In [ ]:
%pip install web3

In [19]:
from web3 import Web3

from web3.middleware import geth_poa_middleware

In [20]:
provider_url = "https://polygon-mumbai.g.alchemy.com/v2/jL3EhrckZEmoV8K0ZQUSiGRQ67PkOakn"

w3 = Web3(Web3.HTTPProvider(provider_url))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# Methods

## Get Current block

### RPC Call

In [17]:
import json
import requests

# Create the RPC Requets dictionary
rpc_request = {
    "jsonrpc" : "2.0",
    "method": "eth_getBlockByNumber",
    "params" : ["latest", True],
    "id": 1
}

response = requests.post(provider_url, data=json.dumps(rpc_request), headers = {'Content-Type': 'application/json'})

if response.status_code == 200:
    data = response.json()
    result = data["result"]
    block_number = int(result["number"],16)
    print(block_number)
else:
    print("RPC Request failed")

41877128


### Web3

In [22]:
print(w3.eth.get_block("latest")["number"])

41877183


## Get Balance

In [23]:
wallet_address = "0xe85fF1BcC7a461D4FE7Ef3a8623d7Eb6a9C0De00"
wallet_pk = "73dcf70b9524d811bb01e7d5cd3cad19834b4d8675219660729aa9d872b94d16"

In [28]:
rpc_request = {
    "jsonrpc" : "2.0",
    "method" : "eth_getBalance",
    "params" : [wallet_address, "latest"],
    "id": 1
}

response = requests.post(provider_url, data=json.dumps(rpc_request), headers = {'Content-Type': 'application/json'})

if response.status_code == 200:
    data = response.json()
    balance_wei = int(data["result"], 16)
    print(balance_wei / 10**18)


0.08442127635846584


In [33]:
balance = w3.eth.get_balance(wallet_address)
scaled_balance = w3.fromWei(balance, "ether")
print(scaled_balance)

0.08442127635846585


In [43]:
from web3.gas_strategies.time_based import fast_gas_price_strategy

# Helper function
def get_balance(address):
    balance = w3.eth.get_balance(address)
    scaled_balance = w3.fromWei(balance, "ether")
    return scaled_balance

recepient_address = "0xc39f29A8766974577658DBF2D687d004Ed4E9c14"

amount_wei = w3.toWei(0.0012314, 'ether')

nonce = w3.eth.get_transaction_count(wallet_address)

w3.eth.setGasPriceStrategy(fast_gas_price_strategy)

transaction = {
    'to' : recepient_address,
    'value' : amount_wei,
    'gas' : 25000,
    "gasPrice" : w3.eth.generateGasPrice(),
    'nonce' : nonce,
    'chainId' : w3.eth.chain_id,
}

from web3 import Account
signed_transaction = Account.sign_transaction(transaction, wallet_pk)
print(signed_transaction)


# Check balance before
print(f"Balance of {wallet_address}: {get_balance(wallet_address)}")
print(f"Balance of {recepient_address}: {get_balance(recepient_address)}")

transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)
print(f"Transaction hash: {transaction_hash.hex()}")


# Check balance after
print(f"Balance of {wallet_address}: {get_balance(wallet_address)}")
print(f"Balance of {recepient_address}: {get_balance(recepient_address)}")


SignedTransaction(rawTransaction=HexBytes('0xf86e03850198b8cc9e8261a894c39f29a8766974577658dbf2d687d004ed4e9c1487045ff3a79390008083027126a0757d07ebcf003c79b093100532eba714b6654dad61057f2dbec3a8d1e9f4e7b2a036a7eabbc9c37a46f583d4d296843c600f6e01740a00a126e815a49292ad64e5'), hash=HexBytes('0x14f6f89b35310828e0fead8f6b0770f9fbbcbd37d1cc69e286994d084a9c3cc2'), r=53141513836951101350756599127327246103229278999521003754727970178613573380018, s=24721577354717736368703750517042241376976963299275050540223863130529523918053, v=160038)
Balance of 0xe85fF1BcC7a461D4FE7Ef3a8623d7Eb6a9C0De00: 0.08283214959080385
Balance of 0xc39f29A8766974577658DBF2D687d004Ed4E9c14: 0.001331400000126
Transaction hash: 0x14f6f89b35310828e0fead8f6b0770f9fbbcbd37d1cc69e286994d084a9c3cc2
Balance of 0xe85fF1BcC7a461D4FE7Ef3a8623d7Eb6a9C0De00: 0.08283214959080385
Balance of 0xc39f29A8766974577658DBF2D687d004Ed4E9c14: 0.001331400000126


In [44]:
from web3.exceptions import TransactionNotFound
import time

#Wait for the transaction to be mined
while True:
    print("Waiting for transaction...")
    try:
        transaction_receipt = w3.eth.get_transaction_receipt(transaction_hash)
        if transaction_receipt is not None:
            if transaction_receipt['status'] == 1:
                print(transaction_receipt)
            else:
                print("Transaction failed.")
            break
        # Wait for a few seconds before checking again
    except TransactionNotFound:
        print("Not Found!")
        time.sleep(0.25)

Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
Not Found!
Waiting for transaction...
AttributeDict({'blockHash': HexBytes('0x8500171a2d29c35b4e4b9ad354741e7ff994a4ea12095885fce2ddaeba9

In [49]:
import time

# https://mumbai.polygonscan.com/tx/0xc2cc024f1a4fd9345b049ef2d93a866dd75bb6b85b6d319c1cec29455c78c5d0#eventlog
event_signature_hash = Web3.keccak(text="Swap(address,address,int256,int256,uint160,uint128,int24)").hex()


block = w3.eth.get_block(41722857)

for tx_hash in  block.transactions: 
    hex_transaction_hash = tx_hash.hex()
    receipt = w3.eth.get_transaction_receipt(hex_transaction_hash)
    for log in receipt.logs:
        log_signature = log["topics"][0].hex()
        if event_signature_hash == log_signature:
            print("Fue un swap")
        else:
            print("No fue un swap")

No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
No fue un swap
Fue un swap
No fue un swap
No fue un swap


In [50]:
v2_router_address = "0x1b02dA8Cb0d097eB8D57A175b88c7D8b47997506"

v2_router_abi = [{"inputs":[{"internalType":"address","name":"_factory","type":"address"},{"internalType":"address","name":"_WETH","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"WETH","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"},{"internalType":"uint256","name":"amountADesired","type":"uint256"},{"internalType":"uint256","name":"amountBDesired","type":"uint256"},{"internalType":"uint256","name":"amountAMin","type":"uint256"},{"internalType":"uint256","name":"amountBMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"addLiquidity","outputs":[{"internalType":"uint256","name":"amountA","type":"uint256"},{"internalType":"uint256","name":"amountB","type":"uint256"},{"internalType":"uint256","name":"liquidity","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amountTokenDesired","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"addLiquidityETH","outputs":[{"internalType":"uint256","name":"amountToken","type":"uint256"},{"internalType":"uint256","name":"amountETH","type":"uint256"},{"internalType":"uint256","name":"liquidity","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"factory","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"reserveIn","type":"uint256"},{"internalType":"uint256","name":"reserveOut","type":"uint256"}],"name":"getAmountIn","outputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"reserveIn","type":"uint256"},{"internalType":"uint256","name":"reserveOut","type":"uint256"}],"name":"getAmountOut","outputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"}],"name":"getAmountsIn","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"}],"name":"getAmountsOut","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountA","type":"uint256"},{"internalType":"uint256","name":"reserveA","type":"uint256"},{"internalType":"uint256","name":"reserveB","type":"uint256"}],"name":"quote","outputs":[{"internalType":"uint256","name":"amountB","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountAMin","type":"uint256"},{"internalType":"uint256","name":"amountBMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"removeLiquidity","outputs":[{"internalType":"uint256","name":"amountA","type":"uint256"},{"internalType":"uint256","name":"amountB","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"removeLiquidityETH","outputs":[{"internalType":"uint256","name":"amountToken","type":"uint256"},{"internalType":"uint256","name":"amountETH","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"removeLiquidityETHSupportingFeeOnTransferTokens","outputs":[{"internalType":"uint256","name":"amountETH","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"bool","name":"approveMax","type":"bool"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"removeLiquidityETHWithPermit","outputs":[{"internalType":"uint256","name":"amountToken","type":"uint256"},{"internalType":"uint256","name":"amountETH","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"bool","name":"approveMax","type":"bool"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"removeLiquidityETHWithPermitSupportingFeeOnTransferTokens","outputs":[{"internalType":"uint256","name":"amountETH","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountAMin","type":"uint256"},{"internalType":"uint256","name":"amountBMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"bool","name":"approveMax","type":"bool"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"removeLiquidityWithPermit","outputs":[{"internalType":"uint256","name":"amountA","type":"uint256"},{"internalType":"uint256","name":"amountB","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapETHForExactTokens","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactETHForTokens","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactETHForTokensSupportingFeeOnTransferTokens","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactTokensForETH","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactTokensForETHSupportingFeeOnTransferTokens","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactTokensForTokens","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactTokensForTokensSupportingFeeOnTransferTokens","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"amountInMax","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapTokensForExactETH","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"amountInMax","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapTokensForExactTokens","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]


v2_router_contract = w3.eth.contract(address=v2_router_address, abi=v2_router_abi)


In [52]:
amount_in = w3.toWei("1", "ether")
reserve_a = w3.toWei("20", "ether")
reserve_b = w3.toWei("1", "ether")

amount_out = v2_router_contract.functions.quote(amount_in, reserve_a, reserve_b).call()
print(w3.fromWei(amount_out,'ether'))

0.05


In [56]:
example_pool = "0x6eDa73f3b08f96A3d1eAc6a892D9Cc96b87F92bd"
v2_pair_abi = [{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"owner","type":"address"},{"indexed":True,"internalType":"address","name":"spender","type":"address"},{"indexed":False,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"sender","type":"address"},{"indexed":False,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":False,"internalType":"uint256","name":"amount1","type":"uint256"},{"indexed":True,"internalType":"address","name":"to","type":"address"}],"name":"Burn","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"sender","type":"address"},{"indexed":False,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":False,"internalType":"uint256","name":"amount1","type":"uint256"}],"name":"Mint","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"sender","type":"address"},{"indexed":False,"internalType":"uint256","name":"amount0In","type":"uint256"},{"indexed":False,"internalType":"uint256","name":"amount1In","type":"uint256"},{"indexed":False,"internalType":"uint256","name":"amount0Out","type":"uint256"},{"indexed":False,"internalType":"uint256","name":"amount1Out","type":"uint256"},{"indexed":True,"internalType":"address","name":"to","type":"address"}],"name":"Swap","type":"event"},{"anonymous":False,"inputs":[{"indexed":False,"internalType":"uint112","name":"reserve0","type":"uint112"},{"indexed":False,"internalType":"uint112","name":"reserve1","type":"uint112"}],"name":"Sync","type":"event"},{"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"from","type":"address"},{"indexed":True,"internalType":"address","name":"to","type":"address"},{"indexed":False,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"MINIMUM_LIQUIDITY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"PERMIT_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"burn","outputs":[{"internalType":"uint256","name":"amount0","type":"uint256"},{"internalType":"uint256","name":"amount1","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"factory","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getReserves","outputs":[{"internalType":"uint112","name":"_reserve0","type":"uint112"},{"internalType":"uint112","name":"_reserve1","type":"uint112"},{"internalType":"uint32","name":"_blockTimestampLast","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_token0","type":"address"},{"internalType":"address","name":"_token1","type":"address"}],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"kLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"mint","outputs":[{"internalType":"uint256","name":"liquidity","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"price0CumulativeLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"price1CumulativeLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"skim","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount0Out","type":"uint256"},{"internalType":"uint256","name":"amount1Out","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"swap","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"sync","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"}]

v2_pair_contract = w3.eth.contract(address=example_pool, abi = v2_pair_abi)

In [58]:
block_num = 41715200

v2_pair_contract.events.Swap().getLogs(
    fromBlock=block_num - 100,
    toBlock = block_num
)

(AttributeDict({'args': AttributeDict({'sender': '0x1b02dA8Cb0d097eB8D57A175b88c7D8b47997506',
   'to': '0x00749ec724946F87138E99961066c9f9a85e1901',
   'amount0In': 0,
   'amount1In': 23000000000000000000,
   'amount0Out': 15851382014759451109,
   'amount1Out': 0}),
  'event': 'Swap',
  'logIndex': 4,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x8ed699d9da6e3be1844d06bd5ca9b3a3e8cd876de0251d766926dd7f9d5c9594'),
  'address': '0x6eDa73f3b08f96A3d1eAc6a892D9Cc96b87F92bd',
  'blockHash': HexBytes('0x8cae3ef40da342f0bd376ba54b7e70f640c18465771a0a8a06e53cedabadfe08'),
  'blockNumber': 41715176}),)